In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

options = Options()
# options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

all_crypto_data = []

try:
    for page in range(1, 4):  # pages 1 to 3
        if page == 1:
            url = "https://coinmarketcap.com/"
        else:
            url = f"https://coinmarketcap.com/?page={page}"
            # Alternative URL format that might work better
            # url = f"https://coinmarketcap.com/rankings/exchanges/{page}/"

        print(f"🔄 Loading Page {page}")
        driver.get(url)
        time.sleep(4)  # Give more time for initial load
        
        # Accept cookies if popup appears
        try:
            accept_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Accept') or contains(text(), 'I agree')]")
            accept_button.click()
            print("✅ Accepted cookies")
            time.sleep(1)
        except:
            pass
        
        # Wait for the table to be present
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table[class*='table']")))
            print(f"✅ Table found on page {page}")
        except:
            print(f"⚠️ Waiting for table on page {page}...")
            time.sleep(3)
        
        # Scroll to load all content
        print("📜 Scrolling to load all data...")
        scroll_pause_time = 1
        screen_height = driver.execute_script("return window.screen.height;")
        i = 1
        
        while True:
            driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
            i += 1
            time.sleep(scroll_pause_time)
            scroll_height = driver.execute_script("return document.body.scrollHeight;")
            if (screen_height * i) > scroll_height:
                break
        
        # One more scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Get page source and parse
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Find the main table - CoinMarketCap uses specific classes
        table = soup.find('table', class_='cmc-table')
        
        if not table:
            # Try alternative table selectors
            table = soup.find('table', {'class': lambda x: x and 'table' in x})
        
        if not table:
            print(f"❌ Could not find table on page {page}")
            # Try to extract data directly from rows
            rows = soup.select('tr[class*="cmc-table-row"]')
            if rows:
                print(f"⚠️ Found {len(rows)} rows using direct selector")
            else:
                continue
        else:
            rows = table.find_all('tr')
        
        # Process rows
        coins_added = 0
        for row in rows:
            try:
                # Skip header rows
                if row.find('th') or not row.find('td'):
                    continue
                
                # Extract data using more specific selectors
                cols = row.find_all('td')
                
                if len(cols) >= 4:  # At least rank, name, price
                    # Get rank
                    rank_elem = row.select_one('td:nth-of-type(2) p')
                    rank = rank_elem.text.strip() if rank_elem else row.select_one('td:nth-of-type(2)').text.strip()
                    
                    # Get name and symbol
                    name_elem = row.select_one('td:nth-of-type(3) p[color="text"]')
                    if not name_elem:
                        name_elem = row.select_one('td:nth-of-type(3) div[class*="name"]')
                    if not name_elem:
                        name_elem = row.select_one('td:nth-of-type(3)')
                    
                    name_text = name_elem.text.strip() if name_elem else ""
                    # Clean name (remove "Buy" text and split name/symbol)
                    name = name_text.replace('Buy', '').strip()
                    
                    # Get price
                    price_elem = row.select_one('td:nth-of-type(4) span')
                    price = price_elem.text.strip() if price_elem else row.select_one('td:nth-of-type(4)').text.strip()
                    
                    # Get 1h change
                    change_1h = "N/A"
                    if len(cols) > 4:
                        change_elem = row.select_one('td:nth-of-type(5) span')
                        if change_elem:
                            change_1h = change_elem.text.strip()
                    
                    # Get 24h change
                    change_24h = "N/A"
                    if len(cols) > 5:
                        change_elem = row.select_one('td:nth-of-type(6) span')
                        if change_elem:
                            change_24h = change_elem.text.strip()
                    
                    # Get 7d change
                    change_7d = "N/A"
                    if len(cols) > 6:
                        change_elem = row.select_one('td:nth-of-type(7) span')
                        if change_elem:
                            change_7d = change_elem.text.strip()
                    
                    # Get market cap
                    market_cap = "N/A"
                    if len(cols) > 7:
                        market_cap = cols[7].text.strip().split('\n')[0] if '\n' in cols[7].text else cols[7].text.strip()
                    
                    # Get volume
                    volume = "N/A"
                    if len(cols) > 8:
                        volume = cols[8].text.strip().split('\n')[0] if '\n' in cols[8].text else cols[8].text.strip()
                    
                    # Get circulating supply
                    supply = "N/A"
                    if len(cols) > 9:
                        supply = cols[9].text.strip()
                    
                    coin_data = {
                        'rank': rank,
                        'name': name,
                        'price': price,
                        '1h_change': change_1h,
                        '24h_change': change_24h,
                        '7d_change': change_7d,
                        'market_cap': market_cap,
                        'volume_24h': volume,
                        'circulating_supply': supply,
                        'page': page
                    }
                    
                    # Only add if we have basic data
                    if coin_data['name'] and coin_data['price']:
                        all_crypto_data.append(coin_data)
                        coins_added += 1
                
            except Exception as e:
                print(f"⚠️ Error processing row: {e}")
                continue
        
        print(f"✅ Page {page}: Added {coins_added} coins")
        
except Exception as e:
    print(f"❌ Error occurred: {e}")
    import traceback
    traceback.print_exc()

finally:
    driver.quit()
    
    print(f"\n✅ Total coins scraped: {len(all_crypto_data)}")
    
    if all_crypto_data:
        # Save to CSV
        df = pd.DataFrame(all_crypto_data)
        df.to_csv('crypto_data.csv', index=False)
        print("💾 Data saved to crypto_data.csv")
        
        # Display first 10 coins
        print("\n📊 First 10 coins:")
        for i, coin in enumerate(all_crypto_data[:10], 1):
            print(f"{i}. {coin['rank']} | {coin['name']:30} | {coin['price']:15} | 24h: {coin['24h_change']:10}")
    else:
        print("❌ No data was scraped")

🔄 Loading Page 1
✅ Table found on page 1
📜 Scrolling to load all data...
✅ Page 1: Added 101 coins
🔄 Loading Page 2
✅ Table found on page 2
📜 Scrolling to load all data...
✅ Page 2: Added 100 coins
🔄 Loading Page 3
✅ Table found on page 3
📜 Scrolling to load all data...
✅ Page 3: Added 100 coins

✅ Total coins scraped: 301
💾 Data saved to crypto_data.csv

📊 First 10 coins:
1.  | CoinMarketCap 20 Index DTFCMC20 | $201.37         | 24h: 0.03%     
2. 1 | BitcoinBTC                     | $95,098.50      | 24h: 0.09%     
3. 2 | EthereumETH                    | $3,319.45       | 24h: 0.69%     
4. 3 | TetherUSDT                     | $0.9996         | 24h: 0.01%     
5. 4 | BNBBNB                         | $944.06         | 24h: 0.16%     
6. 5 | XRPXRP                         | $2.05           | 24h: 0.45%     
7. 6 | SolanaSOL                      | $142.22         | 24h: 1.59%     
8. 7 | USDCUSDC                       | $0.9996         | 24h: 0.00%     
9. 8 | TRONTRX                  

In [11]:
import pyodbc
import pandas as pd

# ================= CONFIG =================
csv_file = 'crypto_data.csv'
server = 'DESKTOP-RLMEU2F'
database = 'cryptoData'

# ============== READ CSV ==================
df = pd.read_csv(csv_file)
print(f"Loaded {len(df)} rows from CSV")

# Clean column names
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

# Rename CSV columns to match SQL table
df.rename(columns={
    '1h_change': 'change_1h',
    '24h_change': 'change_24h',
    '7d_change': 'change_7d'
}, inplace=True)

# Add symbol column if missing
if 'symbol' not in df.columns:
    df['symbol'] = None

# ================== 🔥 REAL FIX ==================

# Convert rank & page safely
df['rank'] = pd.to_numeric(df['rank'], errors='coerce')
df['page'] = pd.to_numeric(df['page'], errors='coerce')

# Convert to pure Python int or None (IMPORTANT)
df['rank'] = df['rank'].apply(lambda x: int(x) if pd.notnull(x) else None)
df['page'] = df['page'].apply(lambda x: int(x) if pd.notnull(x) else None)

# =================================================

# Reorder columns exactly as SQL table expects
df = df[
    [
        'rank',
        'name',
        'symbol',
        'price',
        'change_1h',
        'change_24h',
        'change_7d',
        'market_cap',
        'volume_24h',
        'circulating_supply',
        'page'
    ]
]

# ============ SQL CONNECTION ===============
conn_str = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)

try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    cursor.fast_executemany = True

    data = list(df.itertuples(index=False, name=None))

    insert_sql = """
    INSERT INTO CryptoData
    (rank, name, symbol, price, change_1h, change_24h, change_7d,
     market_cap, volume_24h, circulating_supply, page)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    cursor.executemany(insert_sql, data)
    conn.commit()

    print(f"✅ Successfully inserted {len(data)} records!")

    cursor.execute("SELECT COUNT(*) FROM CryptoData")
    print("Total rows in DB:", cursor.fetchone()[0])

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()


Loaded 301 rows from CSV
✅ Successfully inserted 301 records!
Total rows in DB: 301
